<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/run_e2e_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
run_e2e: convenience runner that executes the full dreamstack pipeline.

This is a thin wrapper around dreamstack_single.cli("pipeline", ...).
It exists to provide a single entry-point with a minimal set of flags.

Usage examples:
  python run_e2e.py
  python run_e2e.py --out-dir out_dreamstack --survivors 8 --frames-per-survivor 12 -vv
  python run_e2e.py --seed 42 --top-n 5 --k 10 --scope global --strategy average --copy
"""

from __future__ import annotations

import argparse
from typing import Optional

from pathlib import Path
import sys

# Import the CLI helper from the main module
try:
    from dreamstack_single import cli
except ImportError:  # allow running when both scripts are in same folder without installation
    # Add current directory to sys.path and try again
    sys.path.insert(0, str(Path(__file__).resolve().parent))
    from dreamstack_single import cli


def build_runner_parser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(
        prog="run_e2e",
        description="Run the full dreamstack pipeline end-to-end.",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
    )
    p.add_argument("-v", "--verbose", action="count", default=1,
                   help="Increase verbosity (-v for INFO, -vv for DEBUG)")

    p.add_argument("--steps", type=int, default=200, help="Simulation steps (placeholder)")
    p.add_argument("--survivors", type=int, default=5, help="Number of survivors to produce")
    p.add_argument("--frames-per-survivor", type=int, default=16, dest="frames_per_survivor",
                   help="Frames per survivor")
    p.add_argument("--out-dir", type=str, default="out_dreamstack", dest="out_dir", help="Output directory")
    p.add_argument("--seed", type=int, default=None, help="Random seed")

    p.add_argument("--top-n", type=int, default=3, dest="top_n", help="Top survivors to carry forward")
    p.add_argument("--k", type=int, default=8, help="Number of motifs")
    p.add_argument("--scope", type=str, choices=("local", "global"), default="local", help="Compression scope")
    p.add_argument("--copy", action="store_true", help="Copy frames into replay dirs")
    p.add_argument("--strategy", type=str, choices=("average", "best", "mix"), default="mix",
                   help="Fusion strategy")

    return p


def main(argv: Optional[list[str]] = None) -> int:
    parser = build_runner_parser()
    ns = parser.parse_args(argv)

    # Pass through to the library CLI without exiting the process
    return cli(
        "pipeline",
        steps=ns.steps,
        survivors=ns.survivors,
        frames_per_survivor=ns.frames_per_survivor,
        out_dir=ns.out_dir,
        seed=ns.seed,
        top_n=ns.top_n,
        k=ns.k,
        scope=ns.scope,
        copy=ns.copy,
        strategy=ns.strategy,
        verbose=ns.verbose,
    )


if __name__ == "__main__":
    raise SystemExit(main())